<center><h1>Extract Transform Load (ETL)</h1></center>

In [ ]:
# Data Manipulation
import pandas as pd
import numpy as np
import warnings

# Database Configuration
from greenhouse_gas_sources import co2_datasets, ch4_datasets, n2o_datasets, sf6_datasets
from noaa_gml_data_extractor import GMLDataExtractor
from dynamodb_data_loader import DynamoDBDataLoader
import boto3

warnings.filterwarnings("ignore") 
pd.set_option('display.max_columns', None)

# AWS tables created specifically for NOAA CO2, CH4, N2O, and SF6 data within the DynamoDB environment.
co2_table = 'CO2DataNOAA'
ch4_table = 'CH4DataNOAA'
n2o_table = 'N2ODataNOAA'
sf6_table = 'SF6DataNOAA'

<center><h1>Carbon Dioxide (CO<sub>2</sub>)</h1></center>

## Extract

In [2]:
extractor = GMLDataExtractor(co2_datasets)
co2_data = extractor.fetch_and_process_co2_data()

print(co2_data.shape)
co2_data.head()

Processing CO2 Data...
Done.
(3581489, 26)


,site_code,year,month,day,hour,minute,second,datetime,time_decimal,midpoint_time,value,value_std_dev,nvalue,latitude,longitude,altitude,elevation,intake_height,qcflag,value_unc,instrument,inst_repeatability,air_sample_container_id,method,event_number,analysis_datetime
0,BRW,1973,7,24,0,0,0,1973-07-24T00:00:00Z,1973.558904,112363200.0,-999.99,-99.99,0.0,71.323,-156.611,1.1,11.0,-9.9,*..,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BRW,1973,7,25,0,0,0,1973-07-25T00:00:00Z,1973.561644,112449600.0,-999.99,-99.99,0.0,71.323,-156.611,1.1,11.0,-9.9,*..,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BRW,1973,7,26,0,0,0,1973-07-26T00:00:00Z,1973.564384,112536000.0,-999.99,-99.99,0.0,71.323,-156.611,1.1,11.0,-9.9,*..,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BRW,1973,7,27,0,0,0,1973-07-27T00:00:00Z,1973.567123,112622400.0,325.21,1.65,10.0,71.323,-156.611,27.0,11.0,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BRW,1973,7,28,0,0,0,1973-07-28T00:00:00Z,1973.569863,112708800.0,324.36,0.23,10.0,71.323,-156.611,27.0,11.0,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Transform

#### Note From NOAA
*qcflag:* This is the NOAA 3-character quality control flag.  Column 1 is the REJECTION flag. An alphanumeric other than a period (.) in the FIRST column indicates a sample with obvious problems during collection or analysis. This measurement should not be interpreted. Column 2 is the SELECTION flag. An alphanumeric other than a period (.) in the SECOND column indicates a sample that is likely valid but does not meet selection criteria determined by the goals of a particular investigation. For example, it might not have been obtained during 'background' conditions.  Column 3 is the INFORMATION flag. An alphanumeric other than a period (.) in the THIRD column provides additional information about the collection or analysis of the sample. A P in the 3rd column of the QC flag indicates the measurement result is preliminary and has not yet been carefully examined by the PI.  The P flag is removed once the quality of the measurement has been determined.

In [ ]:
co2_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3581489 entries, 0 to 3581488
Data columns (total 26 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   site_code                object 
 1   year                     int64  
 2   month                    int64  
 3   day                      int64  
 4   hour                     int64  
 5   minute                   int64  
 6   second                   int64  
 7   datetime                 object 
 8   time_decimal             float64
 9   midpoint_time            float64
 10  value                    float64
 11  value_std_dev            float64
 12  nvalue                   float64
 13  latitude                 float64
 14  longitude                float64
 15  altitude                 float64
 16  elevation                float64
 17  intake_height            float64
 18  qcflag                   object 
 19  value_unc                float64
 20  instrument               object 
 21  inst_rep

#### Dimensionality Reduction

In [ ]:
co2_data = co2_data.drop(columns=['hour', 
                                  'minute', 
                                  'second', 
                                  'time_decimal',
                                  'midpoint_time',
                                  'value_std_dev',
                                  'value_unc',
                                  'nvalue',
                                  'instrument',
                                  'inst_repeatability',
                                  'air_sample_container_id', 
                                  'method', 
                                  'event_number', 
                                  'analysis_datetime']
)

#### Data Cleaning

In [ ]:
# Convert 'datetime' to datetime data type
co2_data['datetime'] = pd.to_datetime(co2_data['datetime'])

# Handling missing values (-999.99 which indicates missing data in this dataset)
co2_data.replace(-999.99, np.NaN, inplace=True)  # replace placeholder with NaN
co2_data.dropna(subset=['value'], inplace=True)  # drop rows where 'value' is NaN

# Filter out rows based on quality control flags if necessary where specifically '...' is not in 'qcflag', and indicates 
# a bad quality measurement
co2_data = co2_data[co2_data['qcflag'].str.match(r'^\.+$', na=False)]

# Sort the data by 'datetime' and 'site_code'
co2_data = co2_data.sort_values(by=['datetime', 'site_code'], ascending=[True, True])
co2_data.reset_index(drop=True, inplace=True)

# Finally, drop duplicates as DynamoDB in AWS does not allow duplicate primary and sort keys
co2_data.drop_duplicates(subset=['datetime', 'site_code'], inplace=True) # primary and sort keys

print(co2_data.shape)

(1217013, 12)


#### Preliminary Feature Engineering

In [ ]:
# Add a season column based on month for seasonal analysis
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'

co2_data['season'] = co2_data['month'].apply(get_season)

# Calculate the rate of change in CO2 levels from one measurement to the next
co2_data['co2_change_rate'] = co2_data.groupby('site_code')['value'].diff().fillna(0)

# Add a new column 'gas' with all entries as 'CO2'
co2_data['gas'] = 'CO2'

co2_data.head()

,site_code,year,month,day,datetime,value,latitude,longitude,altitude,elevation,intake_height,qcflag,season,co2_change_rate,gas
0,NWR,1968,1,16,1968-01-16 20:04:00+00:00,322.36,40.05,-105.63,3526.0,3523.0,3.0,...,Winter,0.00,CO2
1,NWR,1968,1,16,1968-01-16 20:35:00+00:00,322.42,40.05,-105.63,3526.0,3523.0,3.0,...,Winter,0.06,CO2
2,NWR,1968,1,16,1968-01-16 21:00:00+00:00,322.65,40.05,-105.63,3526.0,3523.0,3.0,...,Winter,0.23,CO2
3,NWR,1968,1,16,1968-01-16 21:03:00+00:00,322.46,40.05,-105.63,3526.0,3523.0,3.0,...,Winter,-0.19,CO2
4,NWR,1968,1,16,1968-01-16 21:30:00+00:00,322.56,40.05,-105.63,3526.0,3523.0,3.0,...,Winter,0.10,CO2


#### Renaming & Rearranging

In [ ]:
co2_data.rename(columns = {
    'site_code': 'site', 
    'value': 'ppm'}, 
    inplace = True
)

In [ ]:
# Rearrange the features for enhanced interpretation
co2_data = co2_data[['datetime', 'site', 'ppm', 'latitude', 'longitude', 'altitude', 'elevation', 'intake_height', 'qcflag', 'year', 'month', 'day', 'season', 'co2_change_rate', 'gas']]
print(co2_data.shape)
co2_data.head()

(1217013, 15)


,datetime,site,ppm,latitude,longitude,altitude,elevation,intake_height,qcflag,year,month,day,season,co2_change_rate,gas
0,1968-01-16 20:04:00+00:00,NWR,322.36,40.05,-105.63,3526.0,3523.0,3.0,...,1968,1,16,Winter,0.00,CO2
1,1968-01-16 20:35:00+00:00,NWR,322.42,40.05,-105.63,3526.0,3523.0,3.0,...,1968,1,16,Winter,0.06,CO2
2,1968-01-16 21:00:00+00:00,NWR,322.65,40.05,-105.63,3526.0,3523.0,3.0,...,1968,1,16,Winter,0.23,CO2
3,1968-01-16 21:03:00+00:00,NWR,322.46,40.05,-105.63,3526.0,3523.0,3.0,...,1968,1,16,Winter,-0.19,CO2
4,1968-01-16 21:30:00+00:00,NWR,322.56,40.05,-105.63,3526.0,3523.0,3.0,...,1968,1,16,Winter,0.10,CO2


#### Prepare and Transform Data Structure

In [ ]:
# Initialize, transform, and prepare data for insertion into DynamoDB 
loader = DynamoDBDataLoader(region_name='us-west-2')
all_data_prepared = [loader.prepare_data(row) for index, row in co2_data.iterrows()]

## Load

In [ ]:
# Perform batch writing for the prepared data
loader.batch_write_items(co2_table, all_data_prepared)

'Data upload completed.'

<center><h1>Methane (CH<sub>4</sub>)</h1></center>

## Extract

In [ ]:
extractor = GMLDataExtractor(ch4_datasets)
ch4_data = extractor.fetch_and_process_ch4_data()

print(ch4_data.shape)
ch4_data.head() 

Processing CH4 Data...
Done.
(1238471, 26)


,site_code,year,month,day,hour,minute,second,datetime,time_decimal,midpoint_time,value,value_std_dev,nvalue,latitude,longitude,altitude,elevation,intake_height,qcflag,value_unc,instrument,inst_repeatability,air_sample_container_id,method,event_number,analysis_datetime
0,BRW,1986,1,29,0,0,0,1986-01-29T00:00:00Z,1986.076712,507384000.0,1800.23,27.03,4.0,71.323,-156.611,27.46,11.0,16.46,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BRW,1986,1,30,0,0,0,1986-01-30T00:00:00Z,1986.079452,507470400.0,1787.83,4.12,17.0,71.323,-156.611,27.46,11.0,16.46,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BRW,1986,1,31,0,0,0,1986-01-31T00:00:00Z,1986.082192,507556800.0,1778.05,4.06,7.0,71.323,-156.611,27.46,11.0,16.46,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BRW,1986,2,1,0,0,0,1986-02-01T00:00:00Z,1986.084932,507643200.0,1785.16,4.16,24.0,71.323,-156.611,27.46,11.0,16.46,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BRW,1986,2,2,0,0,0,1986-02-02T00:00:00Z,1986.087671,507729600.0,1795.48,0.00,1.0,71.323,-156.611,27.46,11.0,16.46,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Transform

In [ ]:
ch4_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1238471 entries, 0 to 1238470
Data columns (total 26 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   site_code                1238471 non-null  object 
 1   year                     1238471 non-null  int64  
 2   month                    1238471 non-null  int64  
 3   day                      1238471 non-null  int64  
 4   hour                     1238471 non-null  int64  
 5   minute                   1238471 non-null  int64  
 6   second                   1238471 non-null  int64  
 7   datetime                 1238471 non-null  object 
 8   time_decimal             1238471 non-null  float64
 9   midpoint_time            982591 non-null   float64
 10  value                    1238471 non-null  float64
 11  value_std_dev            982428 non-null   float64
 12  nvalue                   982591 non-null   float64
 13  latitude                 1238471 non-null 

#### Dimensionality Reduction

In [ ]:
ch4_data = ch4_data.drop(columns=['hour', 
                                  'minute', 
                                  'second', 
                                  'time_decimal',
                                  'midpoint_time',
                                  'value_std_dev',
                                  'value_unc',
                                  'nvalue',
                                  'instrument',
                                  'inst_repeatability',
                                  'air_sample_container_id', 
                                  'method', 
                                  'event_number', 
                                  'analysis_datetime']
)

#### Data Cleaning

In [ ]:
# Convert 'datetime' to datetime data type
ch4_data['datetime'] = pd.to_datetime(ch4_data['datetime'])

# Handling missing values (-999.99 which indicates missing data in this dataset)
ch4_data.replace(-999.99, np.NaN, inplace=True)  # replace placeholder with NaN
ch4_data.dropna(subset=['value'], inplace=True)  # drop rows where 'value' is NaN

# Filter out rows based on quality control flags if necessary where specifically '...' is not in 'qcflag', and indicates 
# a bad quality measurement
ch4_data = ch4_data[ch4_data['qcflag'].str.match(r'^\.+$', na=False)]

# Sort the data by 'datetime' and 'site_code'
ch4_data = ch4_data.sort_values(by=['datetime', 'site_code'], ascending=[True, True])
ch4_data.reset_index(drop=True, inplace=True)

# Finally, drop duplicates as DynamoDB in AWS does not allow duplicate primary and sort keys
ch4_data.drop_duplicates(subset=['datetime', 'site_code'], inplace=True) # primary and sort keys

print(ch4_data.shape)

(481653, 12)


#### Preliminary Feature Engineering

In [ ]:
# Add a season column based on month for seasonal analysis
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'

ch4_data['season'] = ch4_data['month'].apply(get_season)

# Calculate the rate of change in CH4 levels from one measurement to the next
ch4_data['ch4_change_rate'] = ch4_data.groupby('site_code')['value'].diff().fillna(0)

# Add a new column 'gas' with all entries as 'CH4'
ch4_data['gas'] = 'CH4'

ch4_data.head()

,site_code,year,month,day,datetime,value,latitude,longitude,altitude,elevation,intake_height,qcflag,season,ch4_change_rate,gas
0,PSA,1983,1,1,1983-01-01 23:25:00+00:00,1556.130,-64.7742,-64.0527,13.0,10.0,3.0,...,Winter,0.000,CH4
1,PSA,1983,1,11,1983-01-11 22:45:00+00:00,1549.043,-64.7742,-64.0527,13.0,10.0,3.0,...,Winter,-7.087,CH4
3,HBA,1983,1,17,1983-01-17 18:30:00+00:00,1560.179,-75.5600,-27.0200,13.0,10.0,3.0,...,Winter,0.000,CH4
5,AMS,1983,1,21,1983-01-21 09:55:00+00:00,1548.031,-37.9500,77.5300,153.0,150.0,3.0,...,Winter,0.000,CH4
7,HBA,1983,1,24,1983-01-24 16:30:00+00:00,1564.229,-75.5600,-27.0200,13.0,10.0,3.0,...,Winter,4.050,CH4


#### Renaming & Rearranging

In [ ]:
ch4_data.rename(columns = {
    'site_code': 'site', 
    'value': 'ppm'}, 
    inplace = True
)

In [ ]:
# Rearrange the features for enhanced interpretation
ch4_data = ch4_data[['datetime', 'site', 'ppm', 'latitude', 'longitude', 'altitude', 'elevation', 'intake_height', 'qcflag', 'year', 'month', 'day', 'season', 'ch4_change_rate', 'gas']]
print(ch4_data.shape)
ch4_data.head()

(481653, 15)


,datetime,site,ppm,latitude,longitude,altitude,elevation,intake_height,qcflag,year,month,day,season,ch4_change_rate,gas
0,1983-01-01 23:25:00+00:00,PSA,1556.130,-64.7742,-64.0527,13.0,10.0,3.0,...,1983,1,1,Winter,0.000,CH4
1,1983-01-11 22:45:00+00:00,PSA,1549.043,-64.7742,-64.0527,13.0,10.0,3.0,...,1983,1,11,Winter,-7.087,CH4
3,1983-01-17 18:30:00+00:00,HBA,1560.179,-75.5600,-27.0200,13.0,10.0,3.0,...,1983,1,17,Winter,0.000,CH4
5,1983-01-21 09:55:00+00:00,AMS,1548.031,-37.9500,77.5300,153.0,150.0,3.0,...,1983,1,21,Winter,0.000,CH4
7,1983-01-24 16:30:00+00:00,HBA,1564.229,-75.5600,-27.0200,13.0,10.0,3.0,...,1983,1,24,Winter,4.050,CH4


#### Prepare and Transform Data Structure

In [ ]:
# Initialize, transform, and prepare data for insertion into DynamoDB 
loader = DynamoDBDataLoader(region_name='us-west-2')
all_ch4_data_prepared = [loader.prepare_data(row) for index, row in ch4_data.iterrows()]

## Load

In [ ]:
# Perform batch writing for the prepared data
loader.batch_write_items(ch4_table, all_ch4_data_prepared)

'Data upload completed.'

<center><h1>Nitrous Oxide (N<sub>2</sub>O)</h1></center>

## Extract

In [2]:
extractor = GMLDataExtractor(n2o_datasets)
n2o_data = extractor.fetch_and_process_n2o_data()

print(n2o_data.shape)
n2o_data.head()

Processing N2O Data...
Done.
(204256, 22)


,site_code,year,month,day,hour,minute,second,datetime,time_decimal,air_sample_container_id,value,value_unc,latitude,longitude,altitude,elevation,intake_height,method,event_number,instrument,analysis_datetime,qcflag
0,ABP,2006,10,27,15,50,0,2006-10-27T15:50:00Z,2006.820986,1779-99,320.22,0.26,-12.76,-38.16,6.0,1.0,5.0,G,231671,H4,2007-04-02T12:30:00,...
1,ABP,2006,10,27,15,50,0,2006-10-27T15:50:00Z,2006.820986,1780-99,320.29,0.26,-12.76,-38.16,6.0,1.0,5.0,G,231672,H4,2007-04-02T12:44:00,...
2,ABP,2006,11,3,16,24,0,2006-11-03T16:24:00Z,2006.840228,6717-66,319.89,0.26,-12.76,-38.16,6.0,1.0,5.0,G,229105,H4,2007-02-20T13:30:00,...
3,ABP,2006,11,3,16,24,0,2006-11-03T16:24:00Z,2006.840228,6718-66,320.40,0.26,-12.76,-38.16,6.0,1.0,5.0,G,229106,H4,2007-02-20T13:44:00,...
4,ABP,2006,11,18,15,47,0,2006-11-18T15:47:00Z,2006.881254,2969-99,319.56,0.26,-12.76,-38.16,6.0,1.0,5.0,G,229103,H4,2007-02-20T13:01:00,...


## Transform

In [3]:
n2o_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204256 entries, 0 to 204255
Data columns (total 22 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   site_code                204256 non-null  object 
 1   year                     204256 non-null  int64  
 2   month                    204256 non-null  int64  
 3   day                      204256 non-null  int64  
 4   hour                     204256 non-null  int64  
 5   minute                   204256 non-null  int64  
 6   second                   204256 non-null  int64  
 7   datetime                 204256 non-null  object 
 8   time_decimal             204256 non-null  float64
 9   air_sample_container_id  204256 non-null  object 
 10  value                    204256 non-null  float64
 11  value_unc                204256 non-null  float64
 12  latitude                 204256 non-null  float64
 13  longitude                204256 non-null  float64
 14  alti

#### Dimensionality Reduction

In [4]:
n2o_data = n2o_data.drop(columns=['hour', 
                                  'minute', 
                                  'second', 
                                  'time_decimal',
                                  'value_unc',
                                  'instrument',
                                  'air_sample_container_id', 
                                  'method', 
                                  'event_number', 
                                  'analysis_datetime']
)

#### Data Cleaning

In [5]:
# Convert 'datetime' to datetime data type
n2o_data['datetime'] = pd.to_datetime(n2o_data['datetime'])

# Handling missing values (-999.99 which indicates missing data in this dataset)
n2o_data.replace(-999.99, np.NaN, inplace=True)  # replace placeholder with NaN
n2o_data.dropna(subset=['value'], inplace=True)  # drop rows where 'value' is NaN

# Filter out rows based on quality control flags if necessary where specifically '...' is not in 'qcflag', and indicates 
# a bad quality measurement
n2o_data = n2o_data[n2o_data['qcflag'].str.match(r'^\.+$', na=False)]

# Sort the data by 'datetime' and 'site_code'
n2o_data = n2o_data.sort_values(by=['datetime', 'site_code'], ascending=[True, True])
n2o_data.reset_index(drop=True, inplace=True)

# Finally, drop duplicates as DynamoDB in AWS does not allow duplicate primary and sort keys
n2o_data.drop_duplicates(subset=['datetime', 'site_code'], inplace=True) # primary and sort keys

print(n2o_data.shape)

(107373, 12)


#### Preliminary Feature Engineering

In [6]:
# Add a season column based on month for seasonal analysis
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'

n2o_data['season'] = n2o_data['month'].apply(get_season)

# Calculate the rate of change in N2O levels from one measurement to the next
n2o_data['n2o_change_rate'] = n2o_data.groupby('site_code')['value'].diff().fillna(0)

# Add a new column 'gas' with all entries as 'N2O'
n2o_data['gas'] = 'N2O'

n2o_data.head()

,site_code,year,month,day,datetime,value,latitude,longitude,altitude,elevation,intake_height,qcflag,season,n2o_change_rate,gas
0,HBA,1996,2,15,1996-02-15 16:55:00+00:00,310.36,-75.605,-26.21,35.0,30.0,5.0,...,Winter,0.00,N2O
1,HBA,1996,3,3,1996-03-03 12:11:00+00:00,310.16,-75.605,-26.21,35.0,30.0,5.0,...,Spring,-0.20,N2O
2,HBA,1996,3,15,1996-03-15 21:14:00+00:00,310.43,-75.605,-26.21,35.0,30.0,5.0,...,Spring,0.27,N2O
3,HBA,1996,3,27,1996-03-27 16:11:00+00:00,310.74,-75.605,-26.21,35.0,30.0,5.0,...,Spring,0.31,N2O
4,HBA,1996,5,15,1996-05-15 12:25:00+00:00,310.24,-75.605,-26.21,35.0,30.0,5.0,...,Spring,-0.50,N2O


#### Renaming & Rearranging

In [7]:
n2o_data.rename(columns = {
    'site_code': 'site', 
    'value': 'ppm'}, 
    inplace = True
)

In [8]:
# Rearrange the features for enhanced interpretation
n2o_data = n2o_data[['datetime', 'site', 'ppm', 'latitude', 'longitude', 'altitude', 'elevation', 'intake_height', 'qcflag', 'year', 'month', 'day', 'season', 'n2o_change_rate', 'gas']]
print(n2o_data.shape)
n2o_data.head()

(107373, 15)


,datetime,site,ppm,latitude,longitude,altitude,elevation,intake_height,qcflag,year,month,day,season,n2o_change_rate,gas
0,1996-02-15 16:55:00+00:00,HBA,310.36,-75.605,-26.21,35.0,30.0,5.0,...,1996,2,15,Winter,0.00,N2O
1,1996-03-03 12:11:00+00:00,HBA,310.16,-75.605,-26.21,35.0,30.0,5.0,...,1996,3,3,Spring,-0.20,N2O
2,1996-03-15 21:14:00+00:00,HBA,310.43,-75.605,-26.21,35.0,30.0,5.0,...,1996,3,15,Spring,0.27,N2O
3,1996-03-27 16:11:00+00:00,HBA,310.74,-75.605,-26.21,35.0,30.0,5.0,...,1996,3,27,Spring,0.31,N2O
4,1996-05-15 12:25:00+00:00,HBA,310.24,-75.605,-26.21,35.0,30.0,5.0,...,1996,5,15,Spring,-0.50,N2O


#### Prepare and Transform Data Structure

In [9]:
# Initialize, transform, and prepare data for insertion into DynamoDB 
loader = DynamoDBDataLoader(region_name='us-west-2')
all_n2o_data_prepared = [loader.prepare_data(row) for index, row in n2o_data.iterrows()]

## Load

In [10]:
# Perform batch writing for the prepared data
loader.batch_write_items(n2o_table, all_n2o_data_prepared)

'Data upload completed.'

<center><h1>Sulfer Hexafluoride (SF<sub>6</sub>)</h1></center>

## Extract

In [2]:
extractor = GMLDataExtractor(sf6_datasets)
sf6_data = extractor.fetch_and_process_sf6_data()

print(sf6_data.shape)
sf6_data.head()

Processing SF6 Data...
Done.
(204213, 22)


,site_code,year,month,day,hour,minute,second,datetime,time_decimal,air_sample_container_id,value,value_unc,latitude,longitude,altitude,elevation,intake_height,method,event_number,instrument,analysis_datetime,qcflag
0,ABP,2006,10,27,15,50,0,2006-10-27T15:50:00Z,2006.820986,1779-99,5.94,0.05,-12.76,-38.16,6.0,1.0,5.0,G,231671,H4,2007-04-02T12:30:00,...
1,ABP,2006,10,27,15,50,0,2006-10-27T15:50:00Z,2006.820986,1780-99,5.88,0.05,-12.76,-38.16,6.0,1.0,5.0,G,231672,H4,2007-04-02T12:44:00,...
2,ABP,2006,11,3,16,24,0,2006-11-03T16:24:00Z,2006.840228,6717-66,6.01,0.05,-12.76,-38.16,6.0,1.0,5.0,G,229105,H4,2007-02-20T13:30:00,...
3,ABP,2006,11,3,16,24,0,2006-11-03T16:24:00Z,2006.840228,6718-66,6.04,0.05,-12.76,-38.16,6.0,1.0,5.0,G,229106,H4,2007-02-20T13:44:00,...
4,ABP,2006,11,18,15,47,0,2006-11-18T15:47:00Z,2006.881254,2969-99,5.85,0.05,-12.76,-38.16,6.0,1.0,5.0,G,229103,H4,2007-02-20T13:01:00,...


## Transform

In [3]:
sf6_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204213 entries, 0 to 204212
Data columns (total 22 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   site_code                204213 non-null  object 
 1   year                     204213 non-null  int64  
 2   month                    204213 non-null  int64  
 3   day                      204213 non-null  int64  
 4   hour                     204213 non-null  int64  
 5   minute                   204213 non-null  int64  
 6   second                   204213 non-null  int64  
 7   datetime                 204213 non-null  object 
 8   time_decimal             204213 non-null  float64
 9   air_sample_container_id  204213 non-null  object 
 10  value                    204213 non-null  float64
 11  value_unc                204213 non-null  float64
 12  latitude                 204213 non-null  float64
 13  longitude                204213 non-null  float64
 14  alti

#### Dimensionality Reduction

In [4]:
sf6_data = sf6_data.drop(columns=['hour', 
                                  'minute', 
                                  'second', 
                                  'time_decimal',
                                  'value_unc',
                                  'instrument',
                                  'air_sample_container_id', 
                                  'method', 
                                  'event_number', 
                                  'analysis_datetime']
)

#### Data Cleaning

In [5]:
# Convert 'datetime' to datetime data type
sf6_data['datetime'] = pd.to_datetime(sf6_data['datetime'])

# Handling missing values (-999.99 which indicates missing data in this dataset)
sf6_data.replace(-999.99, np.NaN, inplace=True)  # replace placeholder with NaN
sf6_data.dropna(subset=['value'], inplace=True)  # drop rows where 'value' is NaN

# Filter out rows based on quality control flags if necessary where specifically '...' is not in 'qcflag', and indicates 
# a bad quality measurement
sf6_data = sf6_data[sf6_data['qcflag'].str.match(r'^\.+$', na=False)]

# Sort the data by 'datetime' and 'site_code'
sf6_data = sf6_data.sort_values(by=['datetime', 'site_code'], ascending=[True, True])
sf6_data.reset_index(drop=True, inplace=True)

# Finally, drop duplicates as DynamoDB in AWS does not allow duplicate primary and sort keys
sf6_data.drop_duplicates(subset=['datetime', 'site_code'], inplace=True) # primary and sort keys

print(sf6_data.shape)

(109469, 12)


#### Preliminary Feature Engineering

In [6]:
# Add a season column based on month for seasonal analysis
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'

sf6_data['season'] = sf6_data['month'].apply(get_season)

# Calculate the rate of change in N2O levels from one measurement to the next
sf6_data['sf6_change_rate'] = sf6_data.groupby('site_code')['value'].diff().fillna(0)

# Add a new column 'gas' with all entries as 'SF6'
sf6_data['gas'] = 'SF6'

sf6_data.head()

,site_code,year,month,day,datetime,value,latitude,longitude,altitude,elevation,intake_height,qcflag,season,sf6_change_rate,gas
0,ASK,1996,12,4,1996-12-04 11:10:00+00:00,3.97,23.2625,5.6322,2715.0,2710.0,5.0,...,Winter,0.00,SF6
1,ASK,1996,12,19,1996-12-19 09:00:00+00:00,3.96,23.2625,5.6322,2715.0,2710.0,5.0,...,Winter,-0.01,SF6
2,ASK,1997,1,4,1997-01-04 12:35:00+00:00,3.90,23.2625,5.6322,2715.0,2710.0,5.0,...,Winter,-0.06,SF6
3,SPO,1997,1,16,1997-01-16 20:57:00+00:00,3.72,-89.9800,-24.8000,2821.3,2810.0,11.3,...,Winter,0.00,SF6
5,SPO,1997,1,16,1997-01-16 23:02:00+00:00,3.80,-89.9800,-24.8000,2815.0,2810.0,5.0,...,Winter,0.08,SF6


#### Renaming & Rearranging

In [7]:
sf6_data.rename(columns = {
    'site_code': 'site', 
    'value': 'ppm'}, 
    inplace = True
)

In [9]:
# Rearrange the features for enhanced interpretation
sf6_data = sf6_data[['datetime', 'site', 'ppm', 'latitude', 'longitude', 'altitude', 'elevation', 'intake_height', 'qcflag', 'year', 'month', 'day', 'season', 'sf6_change_rate', 'gas']]
print(sf6_data.shape)
sf6_data.head()

(109469, 15)


,datetime,site,ppm,latitude,longitude,altitude,elevation,intake_height,qcflag,year,month,day,season,sf6_change_rate,gas
0,1996-12-04 11:10:00+00:00,ASK,3.97,23.2625,5.6322,2715.0,2710.0,5.0,...,1996,12,4,Winter,0.00,SF6
1,1996-12-19 09:00:00+00:00,ASK,3.96,23.2625,5.6322,2715.0,2710.0,5.0,...,1996,12,19,Winter,-0.01,SF6
2,1997-01-04 12:35:00+00:00,ASK,3.90,23.2625,5.6322,2715.0,2710.0,5.0,...,1997,1,4,Winter,-0.06,SF6
3,1997-01-16 20:57:00+00:00,SPO,3.72,-89.9800,-24.8000,2821.3,2810.0,11.3,...,1997,1,16,Winter,0.00,SF6
5,1997-01-16 23:02:00+00:00,SPO,3.80,-89.9800,-24.8000,2815.0,2810.0,5.0,...,1997,1,16,Winter,0.08,SF6


#### Prepare and Transform Data Structure

In [10]:
# Initialize, transform, and prepare data for insertion into DynamoDB 
loader = DynamoDBDataLoader(region_name='us-west-2')
all_sf6_data_prepared = [loader.prepare_data(row) for index, row in sf6_data.iterrows()]

## Load

In [11]:
# Perform batch writing for the prepared data
loader.batch_write_items(sf6_table, all_sf6_data_prepared)

'Data upload completed.'